# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)  

output_data_file

'output_data/cities.csv'

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


605

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
url = "http://api.openweathermap.org/data/2.5/weather?"
# weather_url
units = "standard" 


#get from doc &q
# query_url = f"(url+ appid =(weather_api_key)&units=(unit)&q="
# query_url

query_url = f"{url}appid={weather_api_key}&units={units}&q="

print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=6d987dcb0699091a9eee0d01a310631b&units=standard&q=


In [4]:

# city_response = requests.get(query_url + city)
# city_json = city_response.json()
# city_json = requests.get(query_url + "&q=" + city).json()

# city_json = city_response.json()
# print(city_json)

In [10]:
#lists to hold data needed for plots
City = []
Lon = []
Lat = []
Temp = []
Humidity = []
Clouds = []
Wind = []

# set up retrieval of data from json
#i nclude a print log of each city as it'sbeing processed (with the city number and city name).
 
city_number = 1

for city in cities:
    
#     try:
        final_url = query_url + city
#         
        response = requests.get(final_url).json()
# #       
        City.append(response['sys']['name'])     
        Lon.append(response['coord']['lon'])
        Lat.append(response['coord']['lat'])
        Temp.append(response['main']['temp'])
        Humidity.append(response['main']['humidity'])
        Clouds.append(response['clouds']['all'])
        Wind.append(response['wind']['speed'])
        
        print(f" Processing... {city_number} | {city}")        
        city_number = city_number + 1
        
#     except:
        
# #         print(f" Processing... {city_number} | {city}")
#         print(f" Incomplete Info ... Next Record.")
#         city_number += 1 

#         print(response)

KeyError: 'name'

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#Activity 2-5

# plt.scatter(weather_data["lat"], weather_data["temp"], marker="o")

# plt.title("Latitude vs Temperature")
# plt.ylabel("Temperature(F)")
# plt.xlabel("Latitude")
# plt.grid(True)

# #save the figure
# plt.savefig("Latitude vs Temp")

# plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# plt.scatter(weather_data["lat"], weather_data["temp"], marker="o")

# plt.title("Latitude vs Temperature")
# plt.ylabel("Temperature(F)")
# plt.xlabel("Latitude")
# plt.grid(True)

# #save the figure
# plt.savefig("Latitude vs Temp")

# plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# plt.scatter(weather_data["lat"], weather_data["temp"], marker="o")

# plt.title("Latitude vs Temperature")
# plt.ylabel("Temperature(F)")
# plt.xlabel("Latitude")
# plt.grid(True)

# #save the figure
# plt.savefig("Latitude vs Temp")

# plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# plt.scatter(weather_data["lat"], weather_data["temp"], marker="o")

# plt.title("Latitude vs Temperature")
# plt.ylabel("Temperature(F)")
# plt.xlabel("Latitude")
# plt.grid(True)

# #save the figure
# plt.savefig("Latitude vs Temp")

# plt.show()

## Linear Regression

In [ ]:
# activity 2-7

# # Perform a linear regression on temperature vs. latitude
# (slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# # Get regression values
# regress_values = x_values * slope + intercept
# print(regress_values)


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression